# NLP Ile IMDB Duygu Analizi

### sklearn.feature_extraction.text.CountVectorizer:
Scikit-learn kütüphanesinin metin özellik çıkarımı modülüdür. CountVectorizer sınıfı, metinleri sayısal vektörlere dönüştürmek için kullanılır. Her belge için kelime sayısını hesaplayarak bir belge-terim matrisi oluşturur.
### sklearn.ensemble.RandomForestClassifier:
Scikit-learn kütüphanesinin ensemble (topluluk) öğrenme modülüdür. RandomForestClassifier sınıfı, rastgele orman algoritmasını kullanarak sınıflandırma yapmayı sağlar. Birden çok karar ağacını birleştirerek güçlü ve doğru sınıflandırma modelleri oluşturur.
### bs4.BeautifulSoup:
 BeautifulSoup, HTML ve XML belgelerini işlemek için kullanılan bir Python kütüphanesidir. Web sayfalarından veri kazımak (web scraping) ve belgeleri ayrıştırmak için kullanışlıdır
 ### re:
 Python'un düzenli ifade (regular expression) modülüdür. Metin işleme ve desen eşleme işlemleri için kullanılır. Metinleri belirli kurallara göre filtrelemek, bölmek veya değiştirmek için kullanışlıdır.
 ### nltk:
 NLTK (Natural Language Toolkit), doğal dil işleme için kullanılan bir Python kütüphanesidir. Metin önişleme, tokenizasyon, kelime kökü bulma, durdurma kelimeleri (stopwords) filtreleme gibi dil işleme görevlerini gerçekleştirmek için araçlar sağlar.
 ### nltk.corpus.stopwords:
 NLTK kütüphanesinin corpus (metin koleksiyonu) modülüdür. stopwords alt modülü, yaygın olarak kullanılan durdurma kelimelerinin (stopwords) listesini sağlar. Durdurma kelimeleri, metinlerde sık görülen ancak anlamsal olarak az bilgi taşıyan kelimelerdir (örneğin, "the", "and", "is" gibi). Metin önişleme aşamasında genellikle filtrelenir.

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from bs4 import BeautifulSoup
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

In [38]:
df=pd.read_csv('NLPlabeledData.tsv',delimiter='\t',quoting=3)

In [39]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [40]:
len(df)

25000

In [41]:
# stopwords'ü temizlemek için nltk kütüphanesinden stopwords kelime setini bilgisayarımıza indirmemiz gerekiyor,
# Bu işlemi nltk ile yapıyoruz
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yigit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Veri Temizleme
Veri setinde HTML tagları bulunmakta bunlar modeli şaşırtabilir bunlar temizlenmeli.

In [42]:
sample_review = df.review[0]
sample_review

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [43]:
sample_review=BeautifulSoup(sample_review).get_text()
sample_review
# html taglarından kurtulduk

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [44]:
sample_review=re.sub("[^a-zA-Z]"," ",sample_review)
sample_review
# noktalama işareti ve sayılardan temizle

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    m

In [45]:
sample_review=sample_review.lower()
sample_review
# tüm harfleri küçük yap

' with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    m

In [46]:
# stopwords (yani the, is, ane gibi kelimeler yapay zeka tarafından kullanılmamasını istiyoruz. Bunlar gramer kelimeni.. )
# önce split ite kelimeleri bölüyoruz ve listeye dönüştürüyoruz. amacımız stopwords kelimelerini çıkarmak.
sample_review=sample_review.split()

In [47]:
sample_review

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again',
 'maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent',
 'moonwalker',
 'is',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'which',
 'i',
 'remember',
 'going',
 'to',
 'see',
 'at',
 'the',
 'cinema',
 'when',
 'it',
 'was',
 'originally',
 'released',
 'some',
 'of',
 'it',
 'has',
 'subtle',
 'messages',
 'about',
 'mj',
 's',
 'feeling',
 'towards',
 'the',
 'press',
 'and',
 'also',
 'the',
 'obvious',
 'message',
 'of',
 'drugs',

In [48]:
len(sample_review)

437

In [49]:
swords=set(stopwords.words('english'))
sample_review=[w for w in sample_review if w not in swords]
sample_review

['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 'minutes',
 'excluding',
 'smooth',
 'crim

In [50]:
len(sample_review)

219

bu şekilde bir sample ı temizlemiş olduk şimdi tüm data frame i temizleyelim

In [51]:
def process(review):
    # html taglarından temizlemek için
    review=BeautifulSoup(review).get_text()
    # noktalama işareti ve sayıları kaldırmak için
    review=re.sub("[^a-zA-Z]"," ",review)
    # hepsini küçük harf yap
    review=review.lower()
    review=review.split()
    swords=set(stopwords.words('english'))
    review=[w for w in review if w not in swords]
    return(" ".join(review))

In [52]:
train_x_all= []
for r in range(len(df["review"])):
    if(r+1)%1000==0:
        print("No of reviews processed: ",r+1)
    train_x_all.append(process(df["review"][r]))

C:\Users\yigit\AppData\Local\Temp\ipykernel_17848\3318684267.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review=BeautifulSoup(review).get_text()


No of reviews processed:  1000
No of reviews processed:  2000
No of reviews processed:  3000
No of reviews processed:  4000
No of reviews processed:  5000
No of reviews processed:  6000
No of reviews processed:  7000
No of reviews processed:  8000
No of reviews processed:  9000
No of reviews processed:  10000
No of reviews processed:  11000
No of reviews processed:  12000
No of reviews processed:  13000
No of reviews processed:  14000
No of reviews processed:  15000
No of reviews processed:  16000
No of reviews processed:  17000
No of reviews processed:  18000
No of reviews processed:  19000
No of reviews processed:  20000
No of reviews processed:  21000
No of reviews processed:  22000
No of reviews processed:  23000
No of reviews processed:  24000
No of reviews processed:  25000


In [56]:
x=train_x_all
y=np.array(df["sentiment"])
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.1,random_state=42)

### Bag of Words oluşturuyoruz !

Verilerimizi temizledik ancak yapay zekanın çalışması için bu metin tabanlı verileri sayılara ve bag of words denilen matrise çevirmek gerekiyor. işte bu amaçla sklearn içinde bulunan CountVectorizer aracını kullanıyoruz:

In [57]:
# skleann içinde bulunan countvectorizen fonksiyonunu kullanarak max 5000 kelimelik bag of words oluşturuyoruz.
vectorizer = CountVectorizer(max_features=5000)
train_x=vectorizer.fit_transform(train_x)

## Bag of Words (BoW)
 metin verilerini sayısal vektörlere dönüştürmek için kullanılan bir yaklaşımdır. Bu yaklaşımda, her belge (metin) bir "kelime torbası" olarak temsil edilir ve her kelimenin belgede kaç kez geçtiği sayılır. Belgenin anlamsal yapısı veya kelimelerin sırası dikkate alınmaz, sadece kelimelerin frekansları önemlidir.

Bag of Words'ün temel adımları şunlardır:

1. Kelimeleri Ayırma (Tokenization): Metinler, bireysel kelimelere veya belirteçlere (token) ayrılır. Örneğin, "The quick brown fox jumps over the lazy dog" cümlesi ["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"] şeklinde ayrılır.

2. Sözlük Oluşturma (Vocabulary Creation): Tüm belgeler işlendikten sonra, tüm benzersiz kelimelerden oluşan bir sözlük oluşturulur. Bu sözlük, tüm belgelerde geçen kelimeleri içerir.

3. Kelime Frekansı Hesaplama: Her belge için, sözlükteki her kelimenin o belgede kaç kez geçtiği sayılır. Bu işlem sonucunda, her belge için bir kelime frekans vektörü oluşturulur.

Örneğin, aşağıdaki iki belgeyi ele alalım:
- Belge 1: "The quick brown fox jumps over the lazy dog"
- Belge 2: "A quick brown dog jumps over the lazy fox"

Sözlük: ["a", "brown", "dog", "fox", "jumps", "lazy", "over", "quick", "the"]

Kelime frekans vektörleri:
- Belge 1: [0, 1, 1, 1, 1, 1, 1, 1, 2]
- Belge 2: [1, 1, 1, 1, 1, 1, 1, 1, 1]

Bu vektörler, her belgenin sözlükteki kelimelerin frekanslarını temsil eder. Örneğin, Belge 1'de "the" kelimesi 2 kez geçerken, Belge 2'de 1 kez geçmektedir.

Bag of Words, metinleri sayısal vektörlere dönüştürerek makine öğrenmesi algoritmalarının metinler üzerinde çalışmasını sağlar. Ancak, kelimelerin sırası ve bağlamı kaybolduğu için bazı anlamsal bilgiler de kaybedilir. Ayrıca, seyrek ve yüksek boyutlu vektörler oluşturma eğilimindedir.

Bag of Words, metin sınıflandırma, spam filtreleme, duygu analizi gibi birçok doğal dil işleme (NLP) uygulamasında yaygın olarak kullanılır. Scikit-learn kütüphanesindeki CountVectorizer sınıfı, Bag of Words yaklaşımını uygulayan bir araçtır.

In [58]:
train_x

<22500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1776690 stored elements in Compressed Sparse Row format>

In [59]:
train_x=train_x.toarray()

In [61]:
y_train=train_y
y_train

array([0, 1, 1, ..., 1, 1, 0], dtype=int64)

In [62]:
len(y_train)

22500

## Random Forest Model i oluşturuyoruz ve fit ediyoruz

In [70]:
model=RandomForestClassifier(n_estimators=200)
model.fit(train_x,y_train)

RandomForestClassifier(n_estimators=200)

RandomForestClassifier'daki n_estimators parametresi, modelin kaç adet karar ağacından oluşacağını belirtir. Örneğin, n_estimators=100 demek, rastgele orman modelinin 100 tane bağımsız karar ağacı içereceği anlamına gelir. Her ağaç, verilerin rastgele bir alt kümesi üzerinde eğitilir ve kendi tahminlerini yapar. Sonuçta tüm ağaçların tahminleri birleştirilerek nihai sınıflandırma kararı verilir. Daha fazla ağaç genelde daha iyi performans sağlar ama eğitim süresini de artırır.

In [64]:
test_xx=vectorizer.transform(test_x)

In [65]:
test_xx=test_xx.toarray()

In [68]:
test_xx.shape

(2500, 5000)

In [69]:
test_prediction=model.predict(test_xx)
dogruluk=roc_auc_score(test_y,test_prediction)
print("Doğruluk Oranı : % ",dogruluk * 100)

Doğruluk Oranı : %  85.16314813676684
